Source code: https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=WkAHQrj2Vjbl

In [ ]:
!pip install transformers==2.9.1

In [ ]:
!pip3 install kobert-transformers

In [ ]:
!git clone https://github.com/monologg/KoBERT-Transformers.git
!pip install KoBERT-Transformers/.

fatal: destination path 'KoBERT-Transformers' already exists and is not an empty directory.
Processing ./KoBERT-Transformers
  Created wheel for kobert-transformers: filename=kobert_transformers-0.4.1-cp36-none-any.whl size=12264 sha256=1aee7bde52d729ee91f1383f99e0c871e4683d4fa41ab43c4eb73ea42a085a39
  Stored in directory: /root/.cache/pip/wheels/f8/7c/75/c9ef5a01a4e824bc4689cf444aaa42ce040c680c6f0bb7901e
Successfully built kobert-transformers
  Found existing installation: kobert-transformers 0.4.1
    Uninstalling kobert-transformers-0.4.1:
      Successfully uninstalled kobert-transformers-0.4.1


In [ ]:
from google.colab import files 
src = list(files.upload().values())[0] 
open('tokenization_kobert.py','wb').write(src) # Transformer KoBERT 출처: https://github.com/monologg/KoBERT-Transformers

Saving tokenization_kobert.py to tokenization_kobert (2).py


10994

In [ ]:
import tokenization_kobert

In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

Using TensorFlow backend.


In [ ]:
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [ ]:
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

!pip install -r KoBERT/requirements.txt
!pip install KoBERT/.

fatal: destination path 'KoBERT' already exists and is not an empty directory.
Processing ./KoBERT
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12871 sha256=86ffd3e8c7d1005f063fc87b641060e223a72c83082e6ba028c6a38c94b4320e
  Stored in directory: /tmp/pip-ephem-wheel-cache-n6m_twpo/wheels/d0/ca/3e/bd91a227ac542246f4e8f94ff2f7f12a1a1f0c5b77601bcccf
Successfully built kobert
  Found existing installation: kobert 0.1.1
    Uninstalling kobert-0.1.1:
      Successfully uninstalled kobert-0.1.1


Training set

In [ ]:
sentences = train['document']

In [ ]:
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

In [ ]:
labels = train['label'].values

In [ ]:
from tokenization_kobert import KoBertTokenizer

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [ ]:
MAX_LEN = 64

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)


In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				



In [ ]:
batch_size = 64

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

TEST set

In [ ]:
sentences = test['document']

In [ ]:
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]


In [ ]:
labels = test['label'].values

In [ ]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


In [ ]:
MAX_LEN = 64

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


In [ ]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)


In [ ]:
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)


In [ ]:
batch_size = 64

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
Model

In [ ]:
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

epochs = 5

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

Model Training

In [ ]:
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):

    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

수정-KoBERT

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0

    model.train()
        
    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch
              
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        loss = outputs[0]

        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:

        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...
  Batch   500  of  2,110.    Elapsed: 0:09:33.
  Batch 1,000  of  2,110.    Elapsed: 0:19:12.
  Batch 1,500  of  2,110.    Elapsed: 0:28:48.
  Batch 2,000  of  2,110.    Elapsed: 0:38:28.

  Average training loss: 0.33
  Training epcoh took: 0:40:33

Running Validation...
  Accuracy: 0.89
  Validation took: 0:01:35

======== Epoch 2 / 5 ========
Training...
  Batch   500  of  2,110.    Elapsed: 0:09:35.
  Batch 1,000  of  2,110.    Elapsed: 0:19:14.
  Batch 1,500  of  2,110.    Elapsed: 0:28:48.
  Batch 2,000  of  2,110.    Elapsed: 0:38:25.

  Average training loss: 0.23
  Training epcoh took: 0:40:33

Running Validation...
  Accuracy: 0.90
  Validation took: 0:01:37

======== Epoch 3 / 5 ========
Training...
  Batch   500  of  2,110.    Elapsed: 0:09:42.
  Batch 1,000  of  2,110.    Elapsed: 0:19:24.
  Batch 1,500  of  2,110.    Elapsed: 0:29:06.
  Batch 2,000  of  2,110.    Elapsed: 0:38:49.

  Average training loss: 0.17
  Training epcoh t

TEST

In [ ]:
t0 = time.time()

model.eval()

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for step, batch in enumerate(test_dataloader):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    782.    Elapsed: 0:00:41.
  Batch   200  of    782.    Elapsed: 0:01:23.
  Batch   300  of    782.    Elapsed: 0:02:05.
  Batch   400  of    782.    Elapsed: 0:02:47.
  Batch   500  of    782.    Elapsed: 0:03:29.
  Batch   600  of    782.    Elapsed: 0:04:11.
  Batch   700  of    782.    Elapsed: 0:04:53.

Accuracy: 0.90
Test took: 0:05:27


New Sentences test

In [ ]:
def convert_input_data(sentences):

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    MAX_LEN = 64

    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
def test_sentences(sentences):

    model.eval()

    inputs, masks = convert_input_data(sentences)

    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)

    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving ko_sample.csv to ko_sample.csv


In [ ]:
sample= pd.read_csv('ko_sample.csv', encoding='CP949')


In [ ]:
data= pd.read_csv('ko_sample.csv', encoding='CP949')

수정=KoBERT

In [ ]:
sentences = data['Sentence']
print(sentences[:10])

0                          정말 많이 울었던 영화입니다.
1                                  시간 낭비예요.
2    포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3      지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4                 이걸 영화로 만드는 거야?얼마나 가는지 보자.
5        잔잔한 감동을 주는 영화가 좋은 영화다. 줄리안 무어의 매력!
6                   프랑스 영화, 정말 재미없다는 말밖에...
7                        이보다 더 자연스러울 수는 없다.
8                              잠만 자고 있었는데~~
9             오프닝 씬이... (이 영화와 젊음에 대해 말한다.)
Name: Sentence, dtype: object


In [ ]:
y = sample['Predicted']

for i in range(0,11187):
  logits = test_sentences([sentences[i]])
  y[i] = np.argmax(logits)
  i=i+1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
print (y[:10])

0    1.0
1    0.0
2    1.0
3    1.0
4    0.0
5    1.0
6    0.0
7    1.0
8    0.0
9    1.0
Name: Predicted, dtype: float64


In [ ]:
sample = pd.DataFrame({
    "Id": data["Id"],
    "Predicted": y
})

sample.to_csv('/content/drive/My Drive/sample.csv')